# Create and Run a Simple Network with Caspian

In [2]:
import neuro
import caspian
import risp
import json

### Setup processor

In [3]:
caspian_config = {
    "Leak_Enable": True,
    "Min_Leak": -1,
    "Max_Leak": 4,
    "Min_Axon_Delay": 0,
    "Max_Axon_Delay": 0,
    "Min_Threshold": 0,
    "Max_Threshold": 127,
    "Min_Synapse_Delay": 0,
    "Max_Synapse_Delay": 255,
    "Min_Weight": -127,
    "Max_Weight": 127
}

Cpu = caspian.Processor

npu = Cpu(caspian_config)

Now we can create our network, using the properties from RISP.  This exact process will be followed by every other processor type. To see the details about the network, we can print the network, which will provide the network's json representation.

In [12]:
net = neuro.Network()
# fpath = "experiment_tenn2_tenngineered-milling.json"
# fpath = "../networks/b2_10.json"
# fpath = 'tests/combonet.json'
fpath = "../results/experiment_tenn2_mill20240417_1000t_n10_p100_e1000_s20.json"
with open(fpath) as f:
    j = json.loads(f.read())
net.from_json(j)
processor_params = net.get_data("processor")
app_params = net.get_data("application").to_python()
# net.set_properties(proc.get_network_properties())
# print(net)

npu.load_network(net)
neuro.track_all_output_events(npu, net)

neuro.track_all_neuron_events(npu, net)
net.make_sorted_node_vector()
neuron_ids = [x.id for x in net.sorted_node_vector]

### setup encoders/decoders

In [13]:
def bool_to_one_hot(x: bool):
    return (0, 1) if x else (1, 0)
b2oh = bool_to_one_hot

In [14]:
neuro_tpc = app_params['proc_ticks']
# neuro_tpc = 10
print(neuro_tpc)

encoder_params = {
    "dmin": [0] * 2,  # two bins for each binary input + random
    "dmax": [1] * 2,
    "interval": neuro_tpc,
    "named_encoders": {"s": "spikes"},
    "use_encoders": ["s"] * 2
}
decoder_params = {
    # see notes near where decoder is used
    "dmin": [0] * 4,
    "dmax": [1] * 4,
    "divisor": neuro_tpc,
    "named_decoders": {"r": {"rate": {"discrete": False}}},
    "use_decoders": ["r"] * 4
}
encoder = neuro.EncoderArray(encoder_params)
decoder = neuro.DecoderArray(decoder_params)

spikes = []

10


In [15]:
def run_processor(vector):
    spikes = encoder.get_spikes(vector)
    npu.apply_spikes(spikes)
    npu.run(5)
    npu.run(neuro_tpc)
    
    neuron_counts = npu.neuron_counts()
    data = decoder.get_data_from_processor(npu)
    return neuron_counts, data

In [72]:
spike_train = encoder.get_spikes((0,1))
# [print(s) for s in spike_train]
npu.apply_spikes(spike_train)

In [49]:
print(*run_processor(b2oh(True)))

[0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0.0, 0.0, 0.0, 0.0]


In [48]:
print(*run_processor(b2oh(False)))

[6, 0, 0, 5, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0] [0.0, 0.5, 0.0, 0.0]


In [35]:
spikes = []
spikes.append(neuro.Spike(id=0, time=0, value=1.0))
# spikes.append(neuro.Spike(id=0, time=1, value=0))
# spikes.append(neuro.Spike(id=0, time=0, value=1))
npu.apply_spikes(spikes)

In [61]:
# npu.run(200)

In [67]:
counts = npu.neuron_counts()
vectors = npu.neuron_vectors()
lastfires = npu.neuron_last_fires()
charges = npu.neuron_charges()

data = zip(range(len(counts)), charges, counts, lastfires, vectors, )
data = [['id', 'charge', 'fires', 't_lastfire', 't_fires',]] + list(data)
tabulate.tabulate(data, tablefmt='html')

id,charge,fires,t_lastfire,t_fires
0,0.0,0,-1.0,[]
1,0.0,6,5.0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]"
2,0.0,0,-1.0,[]
3,255.0,4,8.0,"[2.0, 4.0, 6.0, 8.0]"
4,0.0,0,-1.0,[]
5,0.0,2,8.0,"[5.0, 8.0]"
6,0.0,0,-1.0,[]
7,0.0,7,6.0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0]"
8,0.0,5,9.0,"[1.0, 3.0, 5.0, 7.0, 9.0]"
9,61.0,0,-1.0,[]
